### Dataset

##### Examle 1

In [ ]:
from pathlib import Path

In [ ]:
path = Path("./persistence.txt")

In [ ]:
path

PosixPath('persistence.txt')

In [ ]:
data_file = open(path, "rb", buffering=0)

In [ ]:
data_file

<_io.FileIO name='persistence.txt' mode='rb' closefd=True>

In [ ]:
data_file.read(6)

b'persis'

Bring the file cursor back to the beginning of the file

In [ ]:
data_file.seek(0)

0

In [ ]:
data_file.read(6)

b'persis'

##### Example 1.1

In [ ]:
import torch

In [ ]:
number = torch.tensor(6.9, dtype=torch.float32)

In [ ]:
number.numel() * number.element_size()

4

In [ ]:
memory_size = 100_000

In [ ]:
import torch

Create a function that creates a placeholder tensor for a continuous memory block as shown below.

**Hint**: Each number in FP32 format takes up 4 bytes in memory.

In [ ]:
def create_continuous_memory(size):
    FP32_SIZE = 4
    n_numbers = size // FP32_SIZE
    return torch.empty(n_numbers, dtype=torch.float32)

**Explain**

- `size // FP32_SIZE`: It calculates the number of float32 numbers that will fit in the requested size of memory by dividing the size by `FP32_SIZE`

- `torch.empty(n_numbers, dtype=torch.float32)` will create a tensor that takes a continuous block of memory with space for `n_numbers` float32 values.

In [ ]:
cache = create_continuous_memory(memory_size)

In [ ]:
type(cache)

torch.Tensor

In [ ]:
(cache.numel() * cache.element_size()) == memory_size

True

##### Example 1.2

##### Example 1

In [ ]:
import numpy
from typing import List

In [ ]:
# class IndexedCachedDataset:
#     def __init__(self, path):
#         self.data_file = None
#         self.cache_index = None
#         self.path = path
#         self.data_offsets = None
    
#     def read_data(self, path):
#         self.data_file = open(data_file_path(path), "rb", buffering=0)
        
#     def prefetch(self, indicies: List[int]):
#         if all(i in self.cache_index for i in indicies):
#             return
        
#         if not self.data_file:
#             self.read_data(self.path)
        
#         indicies = sorted(set(indicies))
        
#         total_size = 0
#         for i in indicies:
#             total_size += self.data_offsets[i+1] - self.data_offsets[i]
        
#         self.cache = np.empty(total_size, dtype=self.dtype)
#         self.cache_index.clear()
#         ptx = 0
        
#         for i in indicies:
#             self.cache_index[i] = ptx
#             size = self.data_offsets[i+1] - self.data_offsets[i]
#             array = self.cache[ptx:ptx+size]
#             self.data_file.seek()

##### Example 2

In [ ]:
def calculate_memory_size(x):
    return x.numel() * x.element_size()

In [ ]:
data = torch.load("data.pt")

In [ ]:
class CachedDataset(Dataset):
    def __init__(self, filename):
        self.filename = filename
        self.data = None
        self.cache = None
        self.cache_index = {}
    
    def prefetch(self, idxs: List[int]):
        if all(i in self.cache_index for i in idxs):
            return
        
        if not self.data:
            self.data = torch.load(self.filename)
        
        total_elements = sum([self.data[i].numel() for i in idxs])
        self.total_elements = total_elements
        self.cache = torch.zeros(total_elements, dtype=self.data.dtype)
        
        self.cache_index.clear()
        offset = 0
        for i in idxs:
            n_elements = self.data[i].numel()
            self.cache_index[i] = offset
            self.cache[offset:offset+n_elements] = self.data[i].view(-1)
            offset += n_elements

In [ ]:
from torch.utils.data import Dataset

Implement data prefetching for cached data according to the criteria outlined below:

- Check if the next training samples (`idxs`) are already cached (`self.cache_index`).
If they are, skip the prefetching step.
- Verify if the training data is loaded. If it is not, load it before proceeding to load the next training samples into RAM.

In [ ]:
cached_dataset = CachedDataset("data.pt")

In [ ]:
idxs

[69, 42]

In [ ]:
cached_dataset.prefetch(idxs)

In [ ]:
type(cached_dataset.cache)

torch.Tensor

In [ ]:
cached_size = calculate_memory_size(dataset.cache)

In [ ]:
sample_size = sum([calculate_memory_size(data[i]) for i in indices])

In [ ]:
cached_size == sample_size

True

##### Example 3

In [ ]:
import torch